In [1]:
import random
import textwrap
import pandas as pd 
from ipywidgets import widgets, HBox
from pathlib import Path


In [2]:
df = pd.read_json('hotpotqa-cte-inspect.jsonl', lines=True)
print(len(df))
df.head()

7405


,id,question,question_decomposition,context,answer,answers,predicted_triples,predicted_answer,score,n_hops,exact_match,f1,fuzzy_match
0,5a8b57f25542995d1e6f1371,Were Scott Derrickson and Ed Wood of the same ...,"[{'answer': '', 'id': 0, 'paragraph_support_id...",# Scott Derrickson\nScott Derrickson (born Jul...,yes,[yes],"Scott Derrickson, nationality, American\nEd Wo...",Yes,1.000000,2,1,1.000000,0
1,5a8c7595554299585d9e36b6,What government position was held by the woman...,"[{'answer': '', 'id': 0, 'paragraph_support_id...",# Shirley Temple\nShirley Temple Black (April ...,Chief of Protocol,[Chief of Protocol],"Shirley Temple, portrayed, Corliss Archer\nShi...",United States ambassador and Chief of Protocol...,0.461538,3,0,0.461538,0
2,5a85ea095542994775f606a8,"What science fantasy young adult series, told ...","[{'answer': '', 'id': 0, 'paragraph_support_id...",# The Hork-Bajir Chronicles\nThe Hork-Bajir Ch...,Animorphs,[Animorphs],"Animorphs, written by, K. A. Applegate\nThe Ho...",Animorphs,1.000000,5,1,1.000000,1
3,5adbf0a255429947ff17385a,Are the Laleli Mosque and Esma Sultan Mansion ...,"[{'answer': '', 'id': 0, 'paragraph_support_id...","# Laleli Mosque\nThe Laleli Mosque (Turkish: ""...",no,[no],"Laleli Mosque, located_in, Fatih\nEsma Sultan ...",No,1.000000,2,1,1.000000,0
4,5a8e3ea95542995a26add48d,"The director of the romantic comedy ""Big Stone...","[{'answer': '', 'id': 0, 'paragraph_support_id...",# Adriana Trigiani\nAdriana Trigiani is an Ita...,"Greenwich Village, New York City","[Greenwich Village, New York City]","Adriana Trigiani, based in, Greenwich Village\...",New York City,0.750000,2,0,0.750000,0


In [3]:
analysis_path = Path('../tmp/failure-analysis.jsonl')
if analysis_path.exists():
    analysis_df = pd.read_json(analysis_path, lines=True)
    df = df.merge(analysis_df, on='id')
print(len(df))
df.head()

766


,id,question,question_decomposition,context,answer,answers,predicted_triples,predicted_answer,score,n_hops,exact_match,f1,fuzzy_match,analysis
0,5ae0d4c9554299603e418468,Roger O. Egeberg was Assistant Secretary for H...,"[{'answer': '', 'id': 0, 'paragraph_support_id...",# Richard Nixon\nRichard Milhous Nixon (Januar...,1969 until 1974,[1969 until 1974],"Roger O. Egeberg, Assistant Secretary, Nixon a...",1969-1974,0.000000,2,0,0.000000,0,Failure Categories: 4. Incorrect entity or rel...
1,5ac23ff0554299636651994d,"When was Poison's album ""Shut Up, Make Love"" r...","[{'answer': '', 'id': 0, 'paragraph_support_id...",# Poison (American band)\nPoison is an America...,2000,[2000],"Poison, album, Crack a Smile...and More!\nCrac...","There is no album ""Shut Up, Make Love"" by Pois...",0.071429,2,0,0.071429,0,Failure Categories: \n1. Incorrect entity or r...
2,5ae4a3265542995ad6573de5,"Hayden is a singer-songwriter from Canada, but...","[{'answer': '', 'id': 0, 'paragraph_support_id...",# Hayden (musician)\nPaul Hayden Desser (born ...,"Fujioka, Gunma","[Fujioka, Gunma]","Hayden, from, Canada\nBuck-Tick, from, Japan",Japan,0.000000,2,0,0.000000,0,Based on the analysis of the failed Q&A data p...
3,5abf63f15542997ec76fd3ea,Alexander Kerensky was defeated and destroyed ...,"[{'answer': '', 'id': 0, 'paragraph_support_id...",# Socialist Revolutionary Party\nThe Socialist...,October 1922,[October 1922],"Alexander Kerensky, defeated, Bolsheviks\nRuss...",1923,0.000000,2,0,0.000000,0,Failure Categories: \n1. Temporal or hierarchi...
4,5add61d65542995b365fab21,What type of forum did a former Soviet statesm...,"[{'answer': '', 'id': 0, 'paragraph_support_id...",# World Summit of Nobel Peace Laureates\nThe W...,Organizations could come together to address g...,[Organizations could come together to address ...,"Mikhail Gorbachev, initiated, World Summit of ...",a forum,0.000000,2,0,0.000000,0,Failure Categories: \n1. Granularity mismatch\...


In [4]:
success_mask = df['f1'] > 0.3
fail_df = df[~success_mask]
success_df = df[success_mask]
print(len(fail_df), len(success_df), len(df))
print("{:.3f}".format(len(success_df)/len(df)))

766 0 766
0.000


In [5]:
df.groupby('n_hops')[['exact_match', 'f1']].agg(['mean', 'count'])

exact_match              f1      
              mean count      mean count
n_hops                                  
2              0.0   532  0.026929   532
3              0.0   184  0.028675   184
4              0.0    45  0.055681    45
5              0.0     3  0.000000     3
6              0.0     1  0.000000     1
7              0.0     1  0.000000     1

In [6]:
df['triplets'] = df['predicted_triples'].map(str.splitlines)
df['n_triplets'] = df['triplets'].map(len)
df['n_triplets'].describe()

count    766.000000
mean       2.659269
std        0.737483
min        1.000000
25%        2.000000
50%        3.000000
75%        3.000000
max       11.000000
Name: n_triplets, dtype: float64

In [7]:
poor_triplets_mask = df['n_triplets'] < df['n_hops']
df[poor_triplets_mask].describe()

,score,n_hops,exact_match,f1,fuzzy_match,n_triplets
count,107.000000,107.000000,107.0,107.000000,107.000000,107.000000
mean,0.034329,3.495327,0.0,0.034329,0.065421,2.252336
std,0.084568,0.731680,0.0,0.084568,0.248430,0.515669
min,0.000000,2.000000,0.0,0.000000,0.000000,1.000000
25%,0.000000,3.000000,0.0,0.000000,0.000000,2.000000
50%,0.000000,3.000000,0.0,0.000000,0.000000,2.000000
75%,0.000000,4.000000,0.0,0.000000,0.000000,3.000000
max,0.285714,7.000000,0.0,0.285714,1.000000,3.000000


In [8]:
surprise_success_df = df[poor_triplets_mask&success_mask]
len(surprise_success_df)

0

In [9]:
def fixedwidth(text):
    return "\n".join(textwrap.wrap(text, width=80, replace_whitespace=False))

def format_row(row):
    output = []
    output.append(f"{row['id']} - {row['n_hops']} hops")
    output.append("="*80)
    output.append(fixedwidth(row['context']))
    output.append("="*80)
    output.append("Q: " + row['question'])
    output.append("Reference Answers: " + str(row['answers']))
    output.append("Predicted Answer: " + row['predicted_answer'])
    output.append("")
    output.append("# Predicted Triples")
    output.append(row['predicted_triples'])
    output.append("")
    output.append("# Scores")
    output.append("EM: {:.3f}".format(row['exact_match']))
    output.append("F1: {:.3f}".format(row['f1']))
    if 'analysis' in row:
        output.append("")
        output.append("# Analysis")
        output.append(fixedwidth(row['analysis']))
    return "\n".join(output)

def present_row(row):
    print(format_row(row))


def create_browse_app(df):
    def browse_failed(i=0):
        row = df.iloc[i]
        present_row(row)

    index = widgets.IntText(value=0, description='Index:')
    left_button = widgets.Button(description='Previous')
    right_button = widgets.Button(description='Next')

    def on_left_button_clicked(b):
        if index.value > 0:
            index.value -= 1

    def on_right_button_clicked(b):
        if index.value < len(df) - 1:
            index.value += 1

    left_button.on_click(on_left_button_clicked)
    right_button.on_click(on_right_button_clicked)

    ui = HBox([left_button, index, right_button])
    out = widgets.interactive_output(browse_failed, {'i': index})

    display(ui, out)

# Usage

inspect_df = fail_df.sort_values('f1')
create_browse_app(inspect_df)


Output()

In [13]:
present_row(df[df['id']=='5a88ca4e55429938390d3fc2'].iloc[0])

5a88ca4e55429938390d3fc2 - 2 hops
# Duke of Westminster
Duke of Westminster is a title in the Peerage of the
United Kingdom.  It was created by Queen Victoria in 1874 and bestowed upon Hugh
Grosvenor, 3rd Marquess of Westminster.  The 2nd, 3rd, 4th and 5th Dukes were
each his grandsons.  The present holder of the title is Hugh Grosvenor, who
inherited the dukedom on 9 August 2016 following the death of his father,
Gerald.  The present Duke is also a godfather of Prince George of Cambridge.

#
Grosvenor Museum
Grosvenor Museum is a museum in Chester, Cheshire, England.  It
is recorded in the National Heritage List for England as a designated Grade II
listed building.  Its full title is The Grosvenor Museum of Natural History and
Archaeology, with Schools of Science and Art, for Chester, Cheshire and North
Wales.  It takes its name from the family name of the Dukes of Westminster, who
are major landowners in Cheshire.  The museum opened in 1886, it was extended in
1894, and major refurbi